In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [4]:
today = pd.Timestamp('2023-06-29', tz='America/New_York').isoformat()
today

'2023-06-29T00:00:00-04:00'

In [5]:
tickers=["IWV", "IWB", "IWF", "IWD", "IWM", "IWO", "IWN"]

In [9]:
start = pd.Timestamp('2019-07-14', tz='America/New_York').isoformat()
end = pd.Timestamp('2020-07-14', tz='America/New_York').isoformat()

In [10]:
portfolio_df_year = alpaca.get_bars(tickers, timeframe, start=start, end=end).df
portfolio_df_year.tail(10)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2020-06-30 04:00:00+00:00,177.28,180.89,177.2700,180.06,310674,2483,179.380146,IWV
2020-07-01 04:00:00+00:00,180.53,181.84,180.2200,180.98,280723,2167,181.198918,IWV
2020-07-02 04:00:00+00:00,183.29,184.19,181.6600,181.93,149588,1287,182.770857,IWV
2020-07-06 04:00:00+00:00,184.53,185.17,184.0100,184.76,302582,2096,184.532629,IWV
2020-07-07 04:00:00+00:00,183.85,185.14,182.6700,182.78,150832,1358,184.013413,IWV
2020-07-08 04:00:00+00:00,183.40,184.42,182.3400,184.30,342080,2276,183.784628,IWV
2020-07-09 04:00:00+00:00,184.75,184.92,181.0200,183.18,238349,1851,182.998590,IWV
2020-07-10 04:00:00+00:00,183.05,185.28,182.3699,185.13,134397,1296,184.055541,IWV
2020-07-13 04:00:00+00:00,186.52,188.23,182.8600,183.07,240228,1987,185.584792,IWV


In [11]:
IWV=portfolio_df_year[portfolio_df_year['symbol']=='IWV'].drop('symbol', axis=1)
IWB=portfolio_df_year[portfolio_df_year['symbol']=='IWB'].drop('symbol', axis=1)
IWF=portfolio_df_year[portfolio_df_year['symbol']=='IWF'].drop('symbol', axis=1)
IWD=portfolio_df_year[portfolio_df_year['symbol']=='IWD'].drop('symbol', axis=1)
IWM=portfolio_df_year[portfolio_df_year['symbol']=='IWM'].drop('symbol', axis=1)
IWO=portfolio_df_year[portfolio_df_year['symbol']=='IWO'].drop('symbol', axis=1)
IWN=portfolio_df_year[portfolio_df_year['symbol']=='IWN'].drop('symbol', axis=1)
display(IWV.head())
display(IWB.head())
display(IWF.head())
display(IWD.head())
display(IWM.head())
display(IWO.head())
display(IWN.head())

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,176.83,176.8500,176.2800,176.63,238373,1403,176.464251
2019-07-16 04:00:00+00:00,176.53,176.7091,175.9000,176.09,115545,743,176.191883
2019-07-17 04:00:00+00:00,176.09,176.1300,174.9400,174.94,95005,805,175.518579
2019-07-18 04:00:00+00:00,174.60,175.8000,174.4069,175.61,149988,1037,175.270109
2019-07-19 04:00:00+00:00,176.12,176.2183,174.5300,174.53,126205,1153,175.329881


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,167.32,167.46,166.85,167.18,757635,5335,167.036590
2019-07-16 04:00:00+00:00,167.11,167.21,166.42,166.63,1043679,7910,166.736478
2019-07-17 04:00:00+00:00,166.60,166.88,165.52,165.52,578097,4071,166.114306
2019-07-18 04:00:00+00:00,165.24,166.36,165.01,166.19,671214,4051,165.594816
2019-07-19 04:00:00+00:00,166.87,166.87,165.13,165.22,505351,3696,165.861819


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,163.28,163.69,162.900,163.28,1749942,11124,163.126117
2019-07-16 04:00:00+00:00,163.25,163.38,162.400,162.68,765871,8398,162.835240
2019-07-17 04:00:00+00:00,162.82,162.95,161.870,161.87,917000,6871,162.283099
2019-07-18 04:00:00+00:00,161.37,162.43,160.945,162.17,804776,8929,161.763189
2019-07-19 04:00:00+00:00,163.18,163.18,160.935,161.04,1688382,12851,161.613686


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,129.82,129.8700,129.300,129.50,2433075,8452,129.417418
2019-07-16 04:00:00+00:00,129.46,129.5300,129.030,129.18,874563,6124,129.238855
2019-07-17 04:00:00+00:00,129.13,129.1700,128.110,128.12,914867,6339,128.596644
2019-07-18 04:00:00+00:00,128.05,128.9700,127.958,128.80,2546973,14920,128.626073
2019-07-19 04:00:00+00:00,129.12,129.1822,128.200,128.24,1132535,7283,128.749527


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,156.56,156.59,154.85,155.30,12194256,68392,155.363686
2019-07-16 04:00:00+00:00,155.13,156.20,154.67,155.28,15206687,76429,155.526239
2019-07-17 04:00:00+00:00,155.24,155.34,153.73,154.14,14155149,83941,154.449689
2019-07-18 04:00:00+00:00,154.08,155.01,153.73,154.73,15799865,83816,154.394884
2019-07-19 04:00:00+00:00,154.94,155.43,153.82,153.90,17449746,76657,154.656930


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,203.29,203.29,201.62,202.19,348752,3301,202.218619
2019-07-16 04:00:00+00:00,202.21,203.04,201.53,201.90,203953,2834,202.344437
2019-07-17 04:00:00+00:00,201.82,202.09,200.26,200.81,737748,3739,201.370773
2019-07-18 04:00:00+00:00,200.80,202.43,200.58,201.95,1244558,7973,201.733785
2019-07-19 04:00:00+00:00,202.27,202.87,200.47,200.53,165875,2296,201.770682


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-07-15 04:00:00+00:00,121.06,121.1700,119.36,119.65,1412245,6940,119.670036
2019-07-16 04:00:00+00:00,119.68,120.6600,119.15,119.97,849482,4550,119.991401
2019-07-17 04:00:00+00:00,119.89,119.9400,118.50,118.76,695222,5681,119.047690
2019-07-18 04:00:00+00:00,118.76,119.1863,118.33,118.95,437797,4262,118.748936
2019-07-19 04:00:00+00:00,119.22,119.5400,118.60,118.64,484310,4256,119.105296
